# Chess AI Program w/ Monte Carlo Tree Search <br>
Contributors: William Cunningham, David Stephenson, Marty Vo

In [ ]:
# Setting up our necessary imports
import copy
import math
import random
import chess
import chess.svg
from IPython.display import SVG, display

Below are some methods to define the point values of our pieces as well as the methods to actually play the game.

In [ ]:
def display_board(board, white_view):
    """
    Displays the given chess board via SVG. 

    Args:
        board (chess.Board): The board to be displayed.
        white_view (boolean): True if displaying white's POV, False if displaying black's POV
    """
    return SVG(chess.svg.board(board, flipped= not white_view, size=450))

def piece_values(piece_type):
  """
  Returns the value for a chess piece. 
  This heuristic is a common way of ranking the importance of each piece. 
  This is used as a helper to calculate the entire utility of a board state. 

  Args: 
    piece_type: a chess.piece.piece_type. Encodes which chess piece we are looking at. 
  
  Returns: 
    a number representing the value of the given piece_type. 
  """
  if piece_type == chess.PAWN:
    return 1
  elif piece_type == chess.KNIGHT or piece_type == chess.BISHOP:
    return 3 
  elif piece_type == chess.ROOK:
    return 5
  elif piece_type == chess.QUEEN:
    return 9
  else:
    # In Chess, the King has no inherent point value attached to it.
    # Since the King cannot be captured, for our purposes it will hold a value of 10.
    return 10 


def calculate_board_value(board):
  """
  Returns the board utility value for the provided board. 

  Args: 
    board: represents the game board 

  Returns:
    > 0    if WHITE is winning. 
    0      if both WHITE and BLACK have the same utility. 
    < 0    if BLACK is winning. 
  """
  # Getting a dict of pieces from the board 
  piece_map = board.piece_map()
  board_sum = 0
  # Iterating over the map of pieces 
  for piece in piece_map.values():
    # Increment value if WHITE
    if piece.color == chess.WHITE:
      board_sum += piece_values(piece.piece_type)
    # Decrement value if BLACK
    elif piece.color == chess.BLACK:
      board_sum -= piece_values(piece.piece_type)
  return board_sum


def play_player_move(board):
    """
    Gets and plays the player's move for their turn inputted as a string corresponding to chess notation.

    Args:
      board (chess.Board): The board to play the player's move on.

    Returns:
      the board after allowing the player to make a valid move. 
    """

    # Getting a valid move from the user 
    print("Your legal moves are:")
    legal_moves = []
    for move in board.legal_moves:
      legal_moves.append(move.uci())
    middle_index = len(legal_moves) // 2
    print(legal_moves[:middle_index])
    print(legal_moves[middle_index:])
    print("Please enter your move using UCI chess notation:")
    input_string = input()
    print()
    move = None 
    try:
      move = chess.Move.from_uci(input_string)
    except:
      # Invalid move, try getting another move
      print("Invalid move. Try again.")
      return play_player_move(board)
    
    if move not in board.legal_moves:
      print("Invalid move. Try again.")
      return play_player_move(board)
      

    # Updating the board then returning 
    board.push(move)
    return board


def play_ai_move(board, rollout_strategy, simulations, show_node_results):
    """
    Gets and plays the AI's move for their turn.

    Args:
      board (chess.Board): The board to play the AI's move on.
      rollout_strategy (str): Represents which strategy to use for rollouts to end of game.  
      show_node_results (bool): Display the wins and losses of each node.

    Returns:
      the board object after allowing the AI to make its move. 
    """

    # Creating a Node with random rollout startegy 
    game_root = Node(board, None, None, rollout_strategy, board.turn)

    # Running selection for a hardcoded amount of iterations 
    for i in range(simulations):
        game_root.selection()

    best_record = 0
    best_moves = []
    child_winrate = 0

    if show_node_results:
      print("Node move - wins - losses")

    # Traversing over the game_root's children nodes
    for child in game_root.children:
      child_winrate += child.wins
      child_winrate -= child.losses
      if show_node_results:
        print(child.parent_action, child.wins, child.losses)
      if child.wins + child.losses == 0:
        record = 0.5
      else:
        record = child.wins / (child.wins + child.losses)
      if record > best_record:
        best_record = record
        best_moves = []
        best_moves.append(child)
      if record == best_record:
        best_moves.append(child)
    
    if show_node_results:
      print("Differences in wins and losses: " + str(child_winrate))

    # Finding the best child node 
    best_node = best_moves[random.randrange(len(best_moves))]
    # Updating the board then returning 
    board.push(best_node.parent_action)
    return board


def play_game(play_as_white, rollout_strategy, simulations, show_node_results):
    """
    Plays a game of chess until checkmate or stalemate is reached, or until the player resigns.
    The current AI that the player will play against randomly selects a move from the list of valid moves.

    Args:
      play_as_white (bool): Determines whether the player plays as white or black.
      rollout_strategy (str): Represents which strategy to use for rollouts to end of game.  
      show_node_results (bool): Display the wins and losses of each node.
    """
    board = chess.Board()
    i = 0
    for move in board.legal_moves:
        i += 1

    print("Now starting a new game!")
    while True:
        # Display board
        display(display_board(board, play_as_white))
        for i in range(3):
          print("") 
        
        # White's turn
        if board.turn == play_as_white:
            board = play_player_move(board)
        else:
            print("The AI is thinking...")
            board = play_ai_move(board, rollout_strategy, simulations, show_node_results)

        # Check for end of game
        if board.is_game_over():
            break
            
    # Handling end of game 
    if board.is_checkmate():
      outcome = board.result()
      winner = None
      if outcome == "1-0":
          winner = "White"
      else:
          winner = "Black"
      print("Checkmate, {} has won.".format(winner))

    if board.is_stalemate():
      print("Stalemate, the game is a draw.")

    if board.is_insufficient_material():
      print("Insufficient material, the game is a draw.")

Below is the tree data structure for our Monte Carlo Tree Search (MCTS). The Node class also includes the 4 necessary components of MCTS:
1. Selection
2. Expansion
3. Simulation
4. Backpropogation

In [ ]:
class Node(): 
  def __init__(self, state, parent, parent_action, rollout_strat, color):
    """
    Represents a node for usage in MCTS.

    Args: 
      state: the board object representing the state of the game. 
      parent: the Node object this ndoe was derived from. None for root node. 
      parent_action: the acrtion taken by parent Node to get to this state. None for root node. 
      rollout_strat (String): represents which strategy to use for rollouts to end of game.  
    """
    # State is a chess.Board: The board to play the player's move on.
    self.state = state 
    # Reference to the parent Node. 
    self.parent = parent 
    # The move taken to arrive at this state from the parent Node. 
    self.parent_action = parent_action
    # Defining an empty list of children Nodes. 
    self.children = []
    # Number of simulations won/lost/tied from this Node. 
    self.wins, self.losses, self.ties = 0, 0, 0 
    # Setting the rollout startegy for this AI
    self.rollout_strat = rollout_strat
    # Sets which color this AI is playing as
    self.color = color

  def pick_move(self, legal_moves):
    """
    Picks a random, unktaken move from this Node.  

    Args:
      legal_moves: list of Move objects. 

    Returns:
      a single Move object that does not have a Node built for it yet.  
    """
    # List of all actions taken by the current children of this Node. 
    child_actions_taken = []
    for child in self.children:
      # Looking at children's .parent_action field. 
      child_actions_taken.append(child.parent_action)

    # Moves that have not been taken yet 
    untaken_moves = []
    for move in legal_moves:
      # Making sure the move has not been taken by a child of this Node.
      if move not in child_actions_taken:
        untaken_moves.append(move)
    
    rand_index = random.randrange(len(untaken_moves))
    rand_move = untaken_moves[rand_index]
    return rand_move

  def selection(self):
    """
    Select a child node in the tree. If all children have been explored, 
    use the UCB1 formula. Otherwise, expand a random child node. 
    """
    if self.state.legal_moves.count() == 0:
      return 

    # Do the selection
    if self.state.legal_moves.count() == len(self.children):
        

      # We chose to set c as sqrt(2). Does a good job of balancing exploration and exploitation.
      c = 1.414 

      # Get the total number of times we've explored any children.
      total_times_explored = self.wins + self.ties + self.losses

      # Calculate all of the ucb1 scores for each node.
      scores = []
      for node in self.children:
        times_explored = node.wins + node.ties + node.losses
        success_rate = (node.wins + (node.ties / 2)) / times_explored
        ucb1 = success_rate + c * (math.sqrt((2 * math.log(total_times_explored)) / times_explored))
        scores.append(ucb1)
      
      # Get the node which has the highest ucb1 score.
      selection_index = scores.index(max(scores))
      selection_node = self.children[selection_index]

      # Recursively perform selection on the child node we select.
      selection_node.selection()

    else:
      # Do the expansion 
      self.expansion()

  def expansion(self):
    """
    Take a random unexplored move, and create a Node for that next state.
    """
    # Pick a random move.
    new_move = self.pick_move(self.state.legal_moves)
    # Copying the board
    board_copy = copy.deepcopy(self.state)
    board_copy.push(new_move)
    new_board = board_copy
    # Pick a legal move that doesn't have a Node yet.
    # Creating a new node with self as the parent. 
    newNode = Node(new_board, self, new_move, self.rollout_strat, self.color)
    # Do simulation on the new node.
    newNode.simulation()
    # Adding the new node to the children of the parent node.
    self.children.append(newNode)

  def random_rollout(self, board):
    """
    Responsible for picking an available move and doing board.push(move). 
    In this case, the move is completely random for the random rollout strategy. 

    Args:
      board: the representation of the current game board. 
    """
    # Getting a random mpve from all valid moves.
    legal_moves = board.legal_moves
    random_move = random.randrange(legal_moves.count())

    i = 0
    for move in legal_moves:
      if i == random_move:
        # Updating the board.
        board.push(move)
        break
      i += 1

  def attacking_rollout(self, board):
    """
    Out of all the current legal moves, filter for moves that take enemy pieces. 
    Then find the move with the max value, defined by the piece_values function above. 
    This is a fairly common heuristic in chess. Where taking more important pieces result in 
    higher rewards. 

    If there are no attacking moves to play, use random rollout. 

    Args:
      board: the representation of the current game board. 
    """
    legal_moves = board.legal_moves
    max_val = 0
    max_moves_array = []

    for move in legal_moves:
      # Checking where the move lands the piece in question.
      landing_square = move.to_square

      # Checking if the position already has an enemy piece, this represents taking a piece.  
      # piece_val is a number
      piece_val = board.piece_type_at(landing_square)

      # If move is attacking an opponent piece:
      if piece_val is not None:

        # If this is the new max move, update.
        if piece_val > max_val:
          # This is a new high score, need a new moves array.
          max_moves_array = []
          max_val = piece_val
          max_moves_array.append(move) 
        elif piece_val == max_val:
          # This is tie for high score, add to the existing array.
          max_moves_array.append(move)
    
    # If there is no max move, use random rollout.
    if len(max_moves_array) == 0:
      self.random_rollout(board)
    elif len(max_moves_array) == 1:
      # If there is 1 clear best move, commit to it. 
      board.push(max_moves_array[0])
    else:
      # Pick a random move out of the best possible.
      random_index = random.randrange(len(max_moves_array))
      # Commit this random move.
      board.push(max_moves_array[random_index])
  
  def defending_rollout(self, board):
    """
    Out of all the current legal moves, filter for moves that would take a piece out of danger. 
    After finding the moves that could save on eof your pieces from capture, find the move that 
    saves the piece of maximum value. 

    This implementation ensures that move will not take the piece into imediate danger.  
    If there are no defensive moves to take, uses random rollout. 

    Args:
      board: the representation of the current game board. 
    """
    legal_moves = board.legal_moves
    defensive_moves = []
    max_val = 0
    # Iterating over all legal moves.
    for move in legal_moves:
      # Defining where the move starts and ends.
      landing_square = move.to_square
      starting_square = move.from_square
      # Ensuring we move from danger, to safety.
      if board.is_attacked_by(not board.turn, starting_square) and not board.is_attacked_by(
          not board.turn, landing_square):
        # Getting the value for the piece we are considering "saving".
        piece_val = board.piece_type_at(starting_square)
        if piece_val > max_val:
          defensive_moves = []
          max_val = piece_val
          defensive_moves.append(move)
        elif piece_val == max_val:
          defensive_moves.append(move)   
    # If there are no such defensive moves, use random rollout.
    if len(defensive_moves) == 0:
      self.random_rollout(board)
    elif len(defensive_moves) == 1:
      # If there is 1 clear best move, commit to it. 
      board.push(defensive_moves[0])
    else:
      # Pick a random move out of the best possible.
      random_index = random.randrange(len(defensive_moves))
      # Commit this random move. 
      board.push(defensive_moves[random_index])

  def simulation(self):
    """
    Rollout to then end of the game. Utalizes a rollout strategy to make decisions. 
    """
    board = copy.deepcopy(self.state)
    turn = board.turn
    total_moves = 0

    # Loop until the game is over, or we break out after 80 moves.
    while not board.is_game_over():

      total_moves += 1
      # Delegate to the proper rollout strategy. 
      if self.rollout_strat == "random":
        self.random_rollout(board)
      elif self.rollout_strat == "attacking":
        self.attacking_rollout(board)
      elif self.rollout_strat == "defending":
        self.defending_rollout(board)
      # If we have hit the max number of moves, end the simulation to move onto next. 
      if total_moves >= 80:
        break

    # If the number of moves is greater than 80, cancel this simulation.
    if total_moves >= 80:

      # Using helper heuristic to evaluate the board state.
      outcome = calculate_board_value(board)
      if outcome > 0:
        winner = True # WHITE wins 
      elif outcome < 0:
        winner = False # BLACK wins 
      elif outcome == 0:
        winner = None # TIE 

    else:
      outcome = board.result()
      # If the result is a tie:
      if str.startswith(outcome, "1/2"):
        # self.simulation()
        winner = None
      # Win
      elif outcome == "1-0":
        winner = True 
      # Loss
      elif outcome == "0-1":
        winner = False 
    # Use backpropogation so we can "learn" from the previous simulation.
    self.backpropagation(winner)

  def backpropagation(self, winner):
    """
    Traverses up the MCTS Tree from a leaf.

    Args:
      winner: a number representing if white or black won the simulation. Or None if it was a tie.  
    """
    #white = 1, black = 0 
    if winner is None:
      self.ties += 1
    elif winner == self.color:
      self.wins += 1
    else:
      self.losses += 1
    #recursive case 
    if self.parent is not None:
      self.parent.backpropagation(winner)

Now that we have all the necessary components for MCTS, let's try simulating one of our rollout strategies and see what kind of results we get.

In [ ]:
# Creating a Node with random rollout startegy 

# Set up a test board and pieces
test_board = chess.Board()
game_root = Node(test_board, None, None, "attacking", chess.BLACK)
# print(game_root.state)
for i in range(1000):
  game_root.selection()

#print(str(child.wins) + " " + str(child.losses) + " " + str(child.ties))
#i += child.wins + child.losses + child.ties
child_wins = 0
child_losses = 0
for child in game_root.children:
  child_wins += child.wins
  child_losses += child.losses

print("Child wins: " + str(child_wins))
print("Child losses: " + str(child_losses))
print("")
print("Root wins: " + str(game_root.wins))
print("Root losses: " + str(game_root.losses))
print("Root ties: " + str(game_root.ties))

Because we have multiple rollout strategies, let's try to determine which one is the best. This rollout strategy will be the one used in our final AI that players will be able to face off against. Let's focus on comparing our attacking rollout versus our defending rollout.

In [ ]:
def ai_matchup(rollout_1, rollout_2, num_games, simulations):
  """
  Have different rollout methods play each other for the specified number of games.
  Each rollout method will play an equal number of games as White and Black.

  Args:
    rollout_1 (str): The first rollout method.
    rollout_2 (str): The second rollout method.
    num_games (int): The number of games to be played.
    simulations (int): The number of simulations for each move.
  """
  ties = 0
  attacking_rollout_wins = 0
  defending_rollout_wins = 0

  for i in range(num_games):
    board = chess.Board()
    moves_played = 0

    while True:
      # White's turn
      if board.turn == True:
        if i < num_games / 2:
          board = play_ai_move(board, rollout_1, simulations, False)
        else:
          board = play_ai_move(board, rollout_2, simulations, False)
        moves_played += 1
      else:
        if i < num_games / 2:
          board = play_ai_move(board, rollout_2, simulations, False)
        else:
          board = play_ai_move(board, rollout_1, simulations, False)
        moves_played += 1

      # Check for end of game or until 
      if board.is_game_over() or moves_played == 80:
        break

    if board.is_checkmate():
      outcome = board.result()
      winner = None
      if outcome == "1-0":
        if i < num_games / 2:
          attacking_rollout_wins += 1
        else:
          defending_rollout_wins += 1
      else:
        if i < num_games / 2:
          defending_rollout_wins += 1
        else:
          attacking_rollout_wins += 1

    elif board.is_stalemate():
      ties += 1

    elif board.is_insufficient_material():
      ties += 1

    # We've reached the 80 move limit
    else:
      board_score = calculate_board_value(board)
      if board_score > 0:
        attacking_rollout_wins += 1
      elif board_score < 0:
        defending_rollout_wins += 1
      else:
        ties += 1

  print("Attacking rollout wins: {}".format(attacking_rollout_wins))
  print("Defending rollout wins: {}".format(defending_rollout_wins))
  print("Ties: {}".format(ties))

In [ ]:
# Attacking vs defending Rollout
# Note: This may run for a while depending on the number of simulations...
# Results: Attacking: 9 Wins
#          Defending: 15 Wins
ai_matchup("attacking", "defending", 24, 500)

Now that we've determined which rollout is the best, it's time to play some chess! <br>

Below are two different cells which you can run in order to play against our AI. The first cell runs fewer simulations, meaning that the AI will not play as many games during selection. As a result, this AI is not the strongest player, but will respond faster. The following cell runs more simulations, but will take longer to play its move. <br>

You will be playing as White as default. If you'd like to play as Black, you can change the first argument to the ```play_game``` method to ```False```. 

In [ ]:
# Play a (shorter) game as White against our best rollout method
# Note: The input box for the user may not appear due to an issue with Google Colab.
#       If a box to input your move does not appear, please re-run the cell.
play_game(True, "defending", 1000, True)

In [ ]:
# Play a (longer) game as White against our best rollout method
# Note: The input box for the user may not appear due to an issue with Google Colab.
#       If a box to input your move does not appear, please re-run the cell.
play_game(True, "defending", 2000, True)